In [1]:
from init import *

## load shader 'numbers'

In [29]:
NUMBER_FIELDS = [
    "num_views", "num_likes", 
    "num_views_per_day", "num_likes_per_day", 
    "num_passes", "num_characters",
    "num_lines", "num_lines_blank", "num_lines_code", "num_lines_comment",
    "num_chars_code", "num_chars_comment",
]
NUMBER_FIELDS_SHORT = [
    f.replace("num_lines", "nl").replace("num_chars", "nc").replace("num_", "") 
    for f in NUMBER_FIELDS
]
AVERAGE_FIELDS = [
    "num_views_per_day", "num_likes_per_day", "blank_ratio", "code_ratio", "comment_ratio"
]

In [27]:
def get_ratios(df):
    df["blank_ratio"] = df["nl_blank"] / df["nl"]
    df["code_ratio"] = df["nl_code"] / df["nl"]
    df["comment_ratio"] = df["nl_comment"] / df["nl"]

def remove_absolute(df):
    for key in ("characters", "nc_code", "nc_comment", "nl", "nl_blank", "nl_code", "nl_comment"):
        del df[key]

def get_shader_values(qset=None, with_ratio=True, with_absolute=False):
    fields = ["shader_id"] + NUMBER_FIELDS
    
    qset = qset or ShadertoyShader.objects.filter(num_views__gte=0)
    rows = qset.values_list(*fields)
    
    fields = ["shader_id"] + NUMBER_FIELDS_SHORT
    array = np.asarray(rows)
    df = pd.DataFrame(array, columns=fields)
    df.index = df[fields[0]]
    del df[fields[0]]
    for f in fields[1:]:
        df[f] = pd.to_numeric(df[f])
    if with_ratio:
        get_ratios(df)
    if not with_absolute:
        remove_absolute(df)
    return df
values = get_shader_values()
print("num shaders: %s" % len(values))
values

num shaders: 9441


,views,likes,views_per_day,likes_per_day,passes,blank_ratio,code_ratio,comment_ratio
shader_id,,,,,,,,
ls3fD7,437,7,1.908297,0.030568,6,0.169302,0.736495,0.111989
ltKXRy,436,12,0.623748,0.017167,2,0.264706,0.647059,0.088235
4ttXDX,436,4,0.594816,0.005457,3,0.271511,0.697897,0.049713
llK3zt,436,10,0.546366,0.012531,1,0.338028,0.507042,0.154930
XdGfRR,436,24,2.106280,0.115942,2,0.176166,0.694301,0.129534
Msffzs,436,12,0.791289,0.021779,2,0.062992,0.913386,0.440945
4stGWn,435,1,0.390485,0.000898,1,0.357143,0.642857,0.000000
XdVXWd,435,11,0.483333,0.012222,1,0.315789,0.578947,0.157895
XlcGzn,435,9,0.483871,0.010011,1,0.121019,0.777070,0.210191


## correlations

In [23]:
corr = values.corr()
corr.apply(lambda col: col.apply(lambda v: v if abs(v) > .2 else ""))

,views,likes,views_per_day,likes_per_day,passes,blank_ratio,code_ratio,comment_ratio
views,1,0.792292,0.474326,,,,,
likes,0.792292,1,0.426302,0.24279,,,,
views_per_day,0.474326,0.426302,1,0.669088,,,,
likes_per_day,,0.24279,0.669088,1,,,,
passes,,,,,1,,,
blank_ratio,,,,,,1,-0.64066,
code_ratio,,,,,,-0.64066,1,-0.756205
comment_ratio,,,,,,,-0.756205,1


## users

In [30]:
def get_user_stats(qset = None, min_shaders=0, with_ratio=True, with_absolute=False):
    fields = ["username"] + NUMBER_FIELDS
    values = (qset or ShadertoyShader.objects.all()).values_list(*fields)
    
    per_user = dict()
    per_user_count = dict()
    for row in values:
        if row[0] not in per_user:
            per_user[row[0]] = list(row[1:])
            per_user_count[row[0]] = 1
        else:
            for i in range(len(NUMBER_FIELDS)):
                per_user[row[0]][i] += row[i+1] 
            per_user_count[row[0]] += 1
    
    rows = [
        [user] + per_user[user]
        for user in per_user
        if per_user_count[user] > min_shaders
    ]
    
    field_indices = [fields.index(field) for field in AVERAGE_FIELDS if field in fields]
    for row in rows:
        for idx in field_indices:
            row[idx] /= per_user_count[row[0]]
    
    fields = ["username"] + NUMBER_FIELDS_SHORT
    array = np.asarray(rows)
    df = pd.DataFrame(array, columns=fields)
    df.index = df[fields[0]]
    del df[fields[0]]
    for f in fields[1:]:
        df[f] = pd.to_numeric(df[f])
    if with_ratio:
        get_ratios(df)
    if not with_absolute:
        remove_absolute(df)
    return df

users = get_user_stats(min_shaders=10)
users

,views,likes,views_per_day,likes_per_day,passes,blank_ratio,code_ratio,comment_ratio
username,,,,,,,,
ttoinou,4724,66,0.772213,0.010931,14,0.230078,0.408203,0.378125
mahalis,11566,248,0.982775,0.021155,17,0.188755,0.761714,0.078983
iq,3297482,23243,7.654615,0.075112,427,0.211815,0.673977,0.122405
mds2,6537,113,2.606432,0.059181,57,0.186424,0.744722,0.075999
RavenWorks,38197,197,1.926776,0.010176,20,0.308801,0.655687,0.049408
_polymath,4131,73,2.969775,0.063524,23,0.122951,0.778103,0.105386
Klems,41552,862,2.486930,0.064808,64,0.169495,0.717843,0.116585
flockaroo,64015,1937,4.108856,0.128405,97,0.141314,0.745701,0.128730
srtuss,879,51,0.079716,0.005274,27,0.230002,0.717066,0.065037


In [38]:
corr = users.T.corr()
corr.apply(lambda col: col.apply(lambda v: v if abs(v) > .95 else ""))

username,ttoinou,mahalis,iq,mds2,RavenWorks,_polymath,Klems,flockaroo,srtuss,wyatt,...,eiffie,kuvkar,Koltes,nightfox,macbooktall,balkhan,k_kondrak,hopskotchrainbow,BeardThings,DJDoomz
username,,,,,,,,,,,,,,,,,,,,,
ttoinou,1.000000,0.999969,0.999976,0.999982,0.999963,0.999992,0.999974,0.999863,0.998850,0.999592,...,0.999997,0.999953,0.999966,0.999978,0.999995,0.999879,0.999999,0.999931,0.999993,0.999999
mahalis,0.999969,1.000000,0.999900,0.999961,0.999871,0.999982,1.000000,0.999961,0.999085,0.999764,...,0.999983,0.999998,0.999998,0.999896,0.999987,0.999966,0.999957,0.999809,0.999955,0.999965
iq,0.999976,0.999900,1.000000,0.999928,0.999998,0.999941,0.999910,0.999738,0.998502,0.999373,...,0.999966,0.999873,0.999889,0.999987,0.999960,0.999754,0.999986,0.999973,0.999960,0.999971
mds2,0.999982,0.999961,0.999928,1.000000,0.999912,0.999995,0.999965,0.999875,0.999076,0.999678,...,0.999974,0.999948,0.999970,0.999959,0.999973,0.999900,0.999973,0.999901,0.999995,0.999989
RavenWorks,0.999963,0.999871,0.999998,0.999912,1.000000,0.999923,0.999882,0.999692,0.998422,0.999309,...,0.999948,0.999840,0.999859,0.999986,0.999941,0.999711,0.999976,0.999983,0.999948,0.999958
_polymath,0.999992,0.999982,0.999941,0.999995,0.999923,1.000000,0.999985,0.999904,0.999033,0.999689,...,0.999991,0.999971,0.999985,0.999957,0.999991,0.999922,0.999984,0.999894,0.999993,0.999994
Klems,0.999974,1.000000,0.999910,0.999965,0.999882,0.999985,1.000000,0.999955,0.999065,0.999751,...,0.999987,0.999997,0.999997,0.999906,0.999990,0.999961,0.999964,0.999822,0.999961,0.999970
flockaroo,0.999863,0.999961,0.999738,0.999875,0.999692,0.999904,0.999955,1.000000,0.999327,0.999902,...,0.999893,0.999976,0.999964,0.999736,0.999903,0.999997,0.999838,0.999602,0.999847,0.999858
srtuss,0.998850,0.999085,0.998502,0.999076,0.998422,0.999033,0.999065,0.999327,1.000000,0.999727,...,0.998876,0.999136,0.999158,0.998651,0.998900,0.999359,0.998770,0.998403,0.998940,0.998892
